# Arbor Realty Trust Dividend Reinvestment Returns Portfolio Study
#### A simple study of how investing a lump sum in a mREIT would perform based on some past historical data.

In [22]:
import pandas as pd
import numpy as np
import plotly.express as px
import yfinance as yf

#### Initial investment and dividned data from GuruFocus

In [23]:
# Average stock price at purchase date
stockPrice_t0 = 13.486 

# Initial Investment
initialInvestment = 11616 #USD

# Dividend per share at purchase date (quarterly)
dividendPerShare = 0.39 #USD

# Dividend growth rate (yearly, 5y average)
dividendGrowthRate = 0.091 #% 

# Calculate the number of shares purchased at t0
sharesPurchased = initialInvestment / stockPrice_t0

In [24]:
# Obtain the stock data from Yahoo Finance
abr = yf.Ticker("ABR")
stockData = abr.history(period="5y")

# Reset the index
stockData.reset_index(inplace=True)

# Convert 'Date' to date-only format
stockData['Date'] = stockData['Date'].dt.date

In [25]:
# Display the stock price (5Y)
fig = px.line(stockData, x='Date', y='Close', title='ABR Stock Price')
fig.update_xaxes(type='category', nticks=10)
fig.show()

In [26]:
# Calculate daily returns
stockData['Daily_Return'] = stockData['Close'].pct_change()

# Display the daily returns
fig = px.line(stockData, x='Date', y='Daily_Return', title='ABR Daily Returns')
fig.update_xaxes(type='category', nticks=10)
fig.show()

### Now I will calculated the quarterly growth rate and returns, so that it can be used in the simulation for the quarterly dividends

In [27]:
# Calculate volatility (standard deviation of daily returns)
volatility = np.std(stockData['Daily_Return'])

# Calculate growth rate (average daily return)
growthRate = np.mean(stockData['Daily_Return'])

# Number of trading days in a year
tradingDaysInYear = 252

# Number of trading days in a quarter
tradingDaysInQuarter = tradingDaysInYear / 4

# Adjusted quarterly growth rate and volatility
quarterlyGrowthRate = growthRate * tradingDaysInQuarter
quarterlyVolatility = volatility * np.sqrt(tradingDaysInQuarter)

print('Quarterly Volatility: ', quarterlyVolatility)
print('Quarterly Growth Rate: ', quarterlyGrowthRate)

Quarterly Volatility:  0.24309311764214395
Quarterly Growth Rate:  0.05320447316120132


In [28]:
# This is the simulation function

# Define the simulation function
def simulate_dividend_growth(months, 
                             sharesPurchased,
                             initialPrice,
                             quarterlyGrowthRate, # Quarterly growth rate
                             dividendPerShare, 
                             dividendGrowthRate,
                             withholdingTax, 
                             quarterlyVolatility, # Quarterly volatility
                             dividendFrequency=4):
    
    # Convert the yearly dividend growth rate to a quarterly rate
    dividendGrowthRate = (1 + dividendGrowthRate)**(1/4) - 1

    # Create a months array based on the dividend frequency
    monthsArray = np.arange(0, months, 12/dividendFrequency)

    # Initialize the current price and dividend
    currentPrice = initialPrice
    currentDividend = dividendPerShare
    currentShares = sharesPurchased

    # Create a list to store the results
    output = []

    for income in range(1,len(monthsArray)):
        # Calculate the current price by taking into account the growth rate and volatility
        dVolatility = np.random.normal(0, quarterlyVolatility)
        currentPrice = currentPrice * (1 + quarterlyGrowthRate + dVolatility)

        # Calculate the dividend
        currentDividend = currentDividend * (1 + dividendGrowthRate)
        
        # Calculate the dividend income
        dividendIncome = currentDividend * (1 - withholdingTax) * sharesPurchased

        # Increase the number of shares by reinvesting the dividend
        currentShares += dividendIncome / currentPrice
        month = monthsArray[income]

        # Store the results
        output.append({'Month': month, 
                       'Price': currentPrice, 
                       'Dividend': currentDividend, 
                       'Shares': currentShares, 
                       "PortfolioValue":currentPrice*currentShares})

    # Convert the list of dictionaries to a dataframe
    output = pd.DataFrame(output)

    return output

 This function simulates the growth of a dividend-paying stock over a specified number of months. It takes into account factors such as the initial price of the stock, the quarterly growth rate of the stock price, the dividend per share, the growth rate of the dividend, the withholding tax on the dividend, and the quarterly volatility of the stock price.

 The function operates in a loop that runs for the specified number of months. In each iteration, it calculates the current price of the stock by factoring in the growth rate and volatility. It then calculates the current dividend and the dividend income after tax. The dividend income is reinvested by purchasing additional shares of the stock.

 The function keeps track of the month, current price, current dividend, number of shares, and portfolio value at each step and stores these values in a list of dictionaries. Finally, it converts this list into a pandas DataFrame and returns it.

 This simulation can be used to model the potential growth of an investment in a dividend-paying stock over time, taking into account reinvestment of dividends and price volatility.

In [29]:
# Test function
results = simulate_dividend_growth(60, sharesPurchased, stockPrice_t0, quarterlyGrowthRate, dividendPerShare, dividendGrowthRate, 0.15, quarterlyVolatility)
print(results)

    Month      Price  Dividend       Shares  PortfolioValue
0     3.0  15.579469  0.398585   880.068664    13711.002827
1     6.0  22.927325  0.407359   893.076827    20475.862643
2     9.0  22.503646  0.416326   906.621629    20402.291917
3    12.0  12.151193  0.425490   932.258369    11328.051479
4    15.0  12.066713  0.434856   958.642874    11567.668341
5    18.0  13.076417  0.444428   983.526022    12860.996876
6    21.0  13.377739  0.454211  1008.384101    13489.899542
7    24.0  12.622048  0.464210  1035.310400    13067.737518
8    27.0  15.025100  0.474428  1058.428136    15902.988294
9    30.0  15.556182  0.484871  1081.248147    16820.092505
10   33.0   9.225364  0.495545  1120.575212    10337.714350
11   36.0  12.910937  0.506453  1149.294487    14838.468372
12   39.0   9.978417  0.517601  1187.271933    11847.095024
13   42.0  12.393086  0.528995  1218.522948    15101.260162
14   45.0  12.858670  0.540639  1249.305438    16064.406036
15   48.0  13.232875  0.552540  1279.875

Now we'll run the simulation with `n` iterations

In [30]:
# Number of simulation runs
n = 100000

simulationOutcomes = []

# Run the simulation n times
for i in range(n):
    results = simulate_dividend_growth(60, sharesPurchased, stockPrice_t0, quarterlyGrowthRate, dividendPerShare, dividendGrowthRate, 0.15, quarterlyVolatility)
    final = results.iloc[-1]['PortfolioValue']
    simulationOutcomes.append(final)

# Fit a lognormal distribution to the simulation outcomes
mu, std = np.log(np.mean(simulationOutcomes)), np.log(np.std(simulationOutcomes))

# Calculate the median
median = np.exp(mu)

In [32]:
# Display the results as a histogram
fig = px.histogram(x=simulationOutcomes, title='Portfolio Value Distribution After 5 Years of Dividend Reinvestment')
# Calculate the maximum count of the histogram bins
y_max = np.histogram(simulationOutcomes, bins='auto')[0].max()
# Add a vertical line at the median
fig.add_shape(type='line', 
              x0=median, 
              x1=median, 
              y0=0, 
              y1=y_max*0.8, 
              line=dict(color='red', width=3))
fig.add_annotation(x=median, 
                   y=y_max*0.8, 
                   text="Median = {:.2f}".format(median), 
                   showarrow=True, 
                   arrowhead=1, 
                   ax=0, 
                   ay=-50)
# Add a vertical line at the initial investment
fig.add_shape(type='line', 
              x0=initialInvestment, 
              x1=initialInvestment, 
              y0=0, 
              y1=y_max, 
              line=dict(color='green', width=3))
fig.add_annotation(x=initialInvestment, 
                   y=y_max, 
                   text="Initial Investment = {:.2f}".format(initialInvestment), 
                   showarrow=True, 
                   arrowhead=1, 
                   ax=0, 
                   ay=-50)
fig.show()